In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [2]:
# Define the directory containing Excel files
directory = "../../data/"

# Get all Excel files in the directory
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".xlsx")]
column_patterns = ['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ.*', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡ.*ΕΚΔ.*ΠΑΡΑΓ.*', 'ΗΜΕΡ.*ΛΗΞΗ.*ΠΑΡΑΓ.*', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '.*(MW)', 'ΤΕΧΝΟΛ.*'] 

# Sort file paths based on their names to process them in order
file_paths.sort()



In [3]:
def merge_dataframes(df1, df2, column_patterns):
    """
    Merge the first 13 columns of two DataFrames.

    Parameters:
    df1 (DataFrame): First DataFrame.
    df2 (DataFrame): Second DataFrame.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame.
    """
    # Determine the number of columns to merge (minimum between the two DataFrames)
    num_columns_to_merge = max(min(df1.shape[1], df2.shape[1]), len(column_patterns))
    # Select the first 13 columns from each DataFrame
    df1_subset = df1.iloc[:, :num_columns_to_merge]
    df2_subset = df2.iloc[:, :num_columns_to_merge]

    # Merge the subsets along columns
    merged_df = pd.concat([df1_subset, df2_subset], axis=0)

    return merged_df

def merge_dataframes_with_previous_files(file_paths, column_patterns):
    """
    Merge the first 13 columns of DataFrames from multiple Excel files with two sheets each.
    The column names from the first DataFrame are kept.

    Parameters:
    file_paths (list): List of file paths to Excel files.

    Returns:
    DataFrame: Merged DataFrame containing the first 13 columns of each DataFrame from all files.
    """
    merged_df = None
    merged_df_temp = None
    
    # Iterate over each file path
    for file_path in file_paths:
        print(f"\nLoading file : {file_path}")
        # Load Excel file into separate DataFrames
        df = pd.read_excel(file_path, header=1, sheet_name=None, parse_dates=True)
        print(f"Number of sheets detected: {len(df.keys())}")

        # Merge in sheet level
        for sheet_name, df_sheet in df.items():
            # Filter according to pattern columns to keep
            df_sheet = df_sheet.filter(regex='|'.join(column_patterns))
            print(f"Number of columns detected in {sheet_name} : {len(df_sheet.columns)}")
            if merged_df_temp is None:
                # Merge the Sheet DataFrames
                merged_df_temp = df_sheet
            else:
                merged_df_temp = merge_dataframes(merged_df_temp, df_sheet, column_patterns)

        # Merge in file level
        # If merged_df is None, assign it the merged DataFrame, else merge with the previous merged DataFrame
        if merged_df is None:
            merged_df = merged_df_temp
        else:
            merged_df = merge_dataframes(merged_df, merged_df_temp, column_patterns)

    return merged_df

In [4]:
# Merge DataFrames from all Excel files
merged_df = merge_dataframes_with_previous_files(file_paths, column_patterns)


Loading file : ../../data/202012.xlsx
Number of sheets detected: 2
Number of columns detected in ΑΔΕΙΕΣ ΑΠΕ : 11
Number of columns detected in ΑΔΕΙΕΣ ΥΒΡΙΔΙΚΩΝ : 11

Loading file : ../../data/202103.xlsx
Number of sheets detected: 2
Number of columns detected in ΑΔΕΙΕΣ ΑΠΕ : 11
Number of columns detected in ΑΔΕΙΕΣ ΥΒΡΙΔΙΚΩΝ : 11

Loading file : ../../data/202105.xlsx
Number of sheets detected: 2
Number of columns detected in ΑΔΕΙΕΣ ΑΠΕ : 11
Number of columns detected in ΑΔΕΙΕΣ ΥΒΡΙΔΙΚΩΝ : 11

Loading file : ../../data/202108.xlsx
Number of sheets detected: 2
Number of columns detected in ΑΔΕΙΕΣ ΑΠΕ : 11
Number of columns detected in ΑΔΕΙΕΣ ΥΒΡΙΔΙΚΩΝ : 11

Loading file : ../../data/202110.xlsx
Number of sheets detected: 3
Number of columns detected in ΑΔΕΙΕΣ ΑΠΕ : 11
Number of columns detected in ΑΔΕΙΕΣ ΥΒΡΙΔΙΚΩΝ : 11
Number of columns detected in ΑΔΕΙΕΣ ΣΗΘΥΑ : 10

Loading file : ../../data/202203.xlsx
Number of sheets detected: 3
Number of columns detected in ΑΔΕΙΕΣ ΑΠΕ : 11
Number o

In [5]:
merged_df

,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),...,Ισχύς ΑΠΕ (MW),ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ,ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ,ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ,ΤΕΧΝΟΛΟΓΙΑ ΑΠΕ,ΕΓΚΑΤΕΣΤΗΜΕΝΗ ΙΣΧΥΣ ΑΠΕ (MW),ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΕ (MW),ΤΕΧΝΟΛΟΓΙΑ ΑΠΟΘΗΚΕΥΤΙΚΩΝ ΣΥΣΤΗΜΑΤΩΝ,ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΕΓΧΥΣΗΣ (MW),ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΟΡΡΟΦΗΣΗΣ (MW)
0,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,2006-03-23,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,2006-10-02,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.8,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,2006-10-16,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,2009-06-16,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.8,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,NaT,ΕΝΕΡΓΕΙΑΚΗ ΚΟΙΝΟΤΗΤΑ ΠΕΡΙΟΡΙΣΜΕΝΗΣ ΕΥΘΥΝΗΣ ΕΔΕ...,NaT,NaN,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,NaN,ΕΔΕΣΣΑΣ,ΚΟΤΖΑ ΚΙΣΛΑ,NaN,...,NaN,2023-06-09 14:11:50.818000,2023-12-01 09:15:30,2048-12-01 09:15:30,ΦΩΤΟΒΟΛΤΑΪΚΑ,5.73,5.73,ΣΥΣΣΩΡΕΥΤΕΣ,5.73,5.73
116,NaT,ΒΟΛΤΑΛΙΑ ΕΛΛΑΔΟΣ ΑΝΩΝΥΜΟΣ ΕΤΑΙΡΕΙΑ ΑΝΑΝΕΩΣΙΜΩ...,NaT,NaN,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΑΙΤΩΛΟΑΚΑΡΝΑΝΙΑΣ,NaN,ΑΣΤΑΚΟΥ,ΠΟΥΡΝΑΡΙΑ,NaN,...,NaN,2023-06-08 16:00:37.335000,2023-12-01 00:00:00,2048-12-01 00:00:00,ΦΩΤΟΒΟΛΤΑΪΚΑ,93,93,ΣΥΣΣΩΡΕΥΤΕΣ,74.00,74.00
117,NaT,ΒΟΛΤΑΛΙΑ ΕΛΛΑΔΟΣ ΑΝΩΝΥΜΟΣ ΕΤΑΙΡΕΙΑ ΑΝΑΝΕΩΣΙΜΩ...,NaT,NaN,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,ΦΘΙΩΤΙΔΑΣ,NaN,ΞΥΝΙΑΔΑΣ,ΣΠΗΛΙΑ,NaN,...,NaN,2023-10-10 17:19:00,2024-02-19 09:59:00,2049-02-19 00:00:00,ΦΩΤΟΒΟΛΤΑΪΚΑ,6,6,ΣΥΣΣΩΡΕΥΤΕΣ,3.00,3.00
118,NaT,ECOENER HELLAS ΜΟΝΟΠΡΟΣΩΠΗ Α Ε,NaT,NaN,ΘΕΣΣΑΛΙΑΣ,ΜΑΓΝΗΣΙΑΣ,NaN,ΑΛΜΥΡΟΥ,ΑΓΙΟΣ ΙΩΑΝΝΗΣ,NaN,...,NaN,2023-10-09 14:41:00,2024-02-19 10:02:00,2049-02-19 00:00:00,ΦΩΤΟΒΟΛΤΑΪΚΑ,6,6,ΣΥΣΣΩΡΕΥΤΕΣ,20.00,20.00


In [23]:
import re

def is_date(string):
    """
    Check if a string matches common date formats.

    Parameters:
    string (str): Input string.

    Returns:
    bool: True if the string matches a date format, False otherwise.
    """
    # Convert input to string
    string = str(string)

    date_formats = [
        r'\d{4}-\d{1,2}-\d{1,2}',  # YYYY-MM-DD
        r'\d{1,2}-\d{1,2}-\d{4}',  # DD-MM-YYYY
        # Add more date formats as needed
    ]
    for date_format in date_formats:
        if re.match(date_format, string):
            return True
    return False

def convert_to_datetime(df):
    """
    Convert date-like columns in a DataFrame to datetime type.

    Parameters:
    df (DataFrame): Input DataFrame.

    Returns:
    DataFrame: DataFrame with date-like columns converted to datetime type.
    """
    df_copy = df.copy()

    for col in df_copy.columns:
        # Check if the column contains date-like strings
        if df_copy[col].apply(is_date).any():
            try:
                df_copy[col] = pd.to_datetime(df_copy[col], errors='coerce')
                print(f"Column {col} transformed succesfully.\n")
            except:
                print(f"Column {col} is incompatible with datetime.\n")
    return df_copy

In [24]:
def assign_naming(df, colnames=None):

    if colnames is None:
        colnames = ['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΕΤΑΙΡΕΙΑ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
        'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΠΕΡΙΦΕΡΕΙΑ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ',
        'ΔΗΜΟ.* ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΘΕΣΗ', '(MW)', 'ΤΕΧΝΟΛΟΓΙΑ']

    num_columns_to_merge = max(len(df.columns), 13)

    df = df.set_axis(colnames[:num_columns_to_merge], axis='columns')

    return df

In [52]:
# Define pairs of columns to be merged
merge_pairs = {
    'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ': 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ',
    'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
    'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
    'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ': 'ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ', # Adding same assignment column results to overwriting known good values
    # 'ΙΣΧΥΣ (MW)': 'Ισχύς (MW)',
    # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ',
    # 'ΤΕΧΝΟΛΟΓΙΑ' : 'ΤΕΧΝΟΛΟΓΙΑ ΑΠΕ',
    # 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',
    # 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ' : 'ΗΜΕΡ. ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ',
    
}

def merge_columns(df, merge_pairs, inplace=False):
    """
    Merge pairs of columns from a DataFrame where some data is present in one column and some in the other.

    Parameters:
    df (DataFrame): DataFrame to modify.
    merge_pairs (dict): Dictionary where the key is the column to assign values to and the value is the column to delete values from.
    inplace (bool): Whether to modify the DataFrame in-place or return a new DataFrame.

    Returns:
    DataFrame: Modified DataFrame with merged columns.
    """
    if not inplace:
        df = df.copy()

    for col_to_assign, col_to_del in merge_pairs.items():
        if col_to_assign not in df.columns or col_to_del not in df.columns:
            raise ValueError(f"Columns {col_to_assign} or {col_to_del} not found in DataFrame")

        # Filter rows where values exist in the column to delete and the column to assign is NaT or NaN
        rows_with_values = df[col_to_del].notna() & df[col_to_assign].isna()

        # Fill NaN values in the column to assign with values from the column to delete only where values exist and the column to assign is NaT or NaN
        df.loc[rows_with_values, col_to_assign] = df.loc[rows_with_values, col_to_del]

        # Drop the column to delete
        df.drop(columns=col_to_del, inplace=True)

    return df


df_merged_test = convert_to_datetime(merged_df.copy())
# Merge columns based on merge pairs
df_merged_test = merge_columns(df_merged_test.copy(), merge_pairs)

Column ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ transformed succesfully.

Column ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Column ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Column ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Column ΗΜΕΡ. ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ transformed succesfully.

Column ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ transformed succesfully.

Column ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ transformed succesfully.

Column ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ transformed succesfully.



In [53]:
df_merged_test

,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),...,Ισχύς (MW),ΗΜΕΡ. ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,Ισχύς ΑΠΕ (MW),ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ,ΤΕΧΝΟΛΟΓΙΑ ΑΠΕ,ΕΓΚΑΤΕΣΤΗΜΕΝΗ ΙΣΧΥΣ ΑΠΕ (MW),ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΕ (MW),ΤΕΧΝΟΛΟΓΙΑ ΑΠΟΘΗΚΕΥΤΙΚΩΝ ΣΥΣΤΗΜΑΤΩΝ,ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΕΓΧΥΣΗΣ (MW),ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΟΡΡΟΦΗΣΗΣ (MW)
0,2001-02-19 00:00:00.000,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,2003-03-04,2028-03-04 00:00:00,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-11-18 00:00:00.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,2006-03-23,2021-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.0,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2006-01-24 00:00:00.000,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,2006-10-02,2021-10-02 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.8,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-07-22 00:00:00.000,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,2006-10-16,2021-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.0,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,2006-01-24 00:00:00.000,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,2009-06-16,2034-06-16 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.8,...,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023-06-09 14:11:50.818,ΕΝΕΡΓΕΙΑΚΗ ΚΟΙΝΟΤΗΤΑ ΠΕΡΙΟΡΙΣΜΕΝΗΣ ΕΥΘΥΝΗΣ ΕΔΕ...,NaT,2048-12-01 09:15:30,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΠΕΛΛΑΣ,NaN,ΕΔΕΣΣΑΣ,ΚΟΤΖΑ ΚΙΣΛΑ,NaN,...,NaN,NaT,NaN,2023-12-01 09:15:30,ΦΩΤΟΒΟΛΤΑΪΚΑ,5.73,5.73,ΣΥΣΣΩΡΕΥΤΕΣ,5.73,5.73
116,2023-06-08 16:00:37.335,ΒΟΛΤΑΛΙΑ ΕΛΛΑΔΟΣ ΑΝΩΝΥΜΟΣ ΕΤΑΙΡΕΙΑ ΑΝΑΝΕΩΣΙΜΩ...,NaT,2048-12-01 00:00:00,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΑΙΤΩΛΟΑΚΑΡΝΑΝΙΑΣ,NaN,ΑΣΤΑΚΟΥ,ΠΟΥΡΝΑΡΙΑ,NaN,...,NaN,NaT,NaN,2023-12-01 00:00:00,ΦΩΤΟΒΟΛΤΑΪΚΑ,93,93,ΣΥΣΣΩΡΕΥΤΕΣ,74.00,74.00
117,2023-10-10 17:19:00.000,ΒΟΛΤΑΛΙΑ ΕΛΛΑΔΟΣ ΑΝΩΝΥΜΟΣ ΕΤΑΙΡΕΙΑ ΑΝΑΝΕΩΣΙΜΩ...,NaT,2049-02-19 00:00:00,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,ΦΘΙΩΤΙΔΑΣ,NaN,ΞΥΝΙΑΔΑΣ,ΣΠΗΛΙΑ,NaN,...,NaN,NaT,NaN,2024-02-19 09:59:00,ΦΩΤΟΒΟΛΤΑΪΚΑ,6,6,ΣΥΣΣΩΡΕΥΤΕΣ,3.00,3.00
118,2023-10-09 14:41:00.000,ECOENER HELLAS ΜΟΝΟΠΡΟΣΩΠΗ Α Ε,NaT,2049-02-19 00:00:00,ΘΕΣΣΑΛΙΑΣ,ΜΑΓΝΗΣΙΑΣ,NaN,ΑΛΜΥΡΟΥ,ΑΓΙΟΣ ΙΩΑΝΝΗΣ,NaN,...,NaN,NaT,NaN,2024-02-19 10:02:00,ΦΩΤΟΒΟΛΤΑΪΚΑ,6,6,ΣΥΣΣΩΡΕΥΤΕΣ,20.00,20.00


In [47]:
df_merged_test.dtypes

ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ            datetime64[ns]
ΕΤΑΙΡΕΙΑ                                       object
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ           datetime64[ns]
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ          datetime64[ns]
ΠΕΡΙΦΕΡΕΙΑ                                     object
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ                           object
ΔΗΜΟΣ                                          object
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                               object
ΘΕΣΗ                                           object
ΙΣΧΥΣ (MW)                                    float64
ΤΕΧΝΟΛΟΓΙΑ                                     object
Ισχύς ΑΠΕ (MW)                                float64
ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ      datetime64[ns]
ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ        datetime64[ns]
ΕΓΚΑΤΕΣΤΗΜΕΝΗ ΙΣΧΥΣ ΑΠΕ  (MW)                  object
ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΕ (MW)                         object
ΤΕΧΝΟΛΟΓΙΑ ΑΠΟΘΗΚΕΥΤΙΚΩΝ ΣΥΣΤΗΜΑΤΩΝ            object
ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΕΓΧΥΣΗΣ (MW)                    float64
ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΟΡΡΟΦΗΣΗΣ (M

In [38]:
df_merged_test.isna().sum()

ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ             2753
ΕΤΑΙΡΕΙΑ                                 283
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ           36337
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ          37078
ΠΕΡΙΦΕΡΕΙΑ                               283
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ                   33893
ΔΗΜΟΣ                                  37021
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                         388
ΘΕΣΗ                                     283
ΙΣΧΥΣ (MW)                             37035
ΤΕΧΝΟΛΟΓΙΑ                             19681
Ισχύς ΑΠΕ (MW)                         73960
ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ      73343
ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ        73373
ΕΓΚΑΤΕΣΤΗΜΕΝΗ ΙΣΧΥΣ ΑΠΕ  (MW)          73433
ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΕ (MW)                 73463
ΤΕΧΝΟΛΟΓΙΑ ΑΠΟΘΗΚΕΥΤΙΚΩΝ ΣΥΣΤΗΜΑΤΩΝ    73493
ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΕΓΧΥΣΗΣ (MW)             73526
ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΟΡΡΟΦΗΣΗΣ (MW)         73866
dtype: int64

In [39]:
merged_df.isna().sum()

ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ             3426
ΕΤΑΙΡΕΙΑ                                 283
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ           37021
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ          37021
ΠΕΡΙΦΕΡΕΙΑ                               283
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ                   33893
ΔΗΜΟΣ                                  37021
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                         388
ΘΕΣΗ                                     283
ΙΣΧΥΣ (MW)                             37036
ΤΕΧΝΟΛΟΓΙΑ                             20264
ΗΜΕΡ. ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ                73302
Ισχύς (MW)                             73985
ΗΜΕΡ. ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ               73959
Ισχύς ΑΠΕ (MW)                         73960
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ                    73313
ΗΜΕΡ. ΕΚΔΟΣΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ      73343
ΗΜΕΡ. ΛΗΞΗΣ ΒΕΒΑΙΩΣΗΣ ΠΑΡΑΓΩΓΟΥ        73373
ΤΕΧΝΟΛΟΓΙΑ ΑΠΕ                         73403
ΕΓΚΑΤΕΣΤΗΜΕΝΗ ΙΣΧΥΣ ΑΠΕ  (MW)          73433
ΜΕΓΙΣΤΗ ΙΣΧΥΣ ΑΠΕ (MW)                 73463
ΤΕΧΝΟΛΟΓΙΑ ΑΠΟΘΗΚΕΥΤΙΚΩΝ ΣΥΣΤΗΜΑΤΩΝ    73493
ΜΕΓΙΣΤΗ ΙΣ

In [12]:
# sample_df = merge_dataframes(sample_df[0], sample_df[1])

In [18]:
# sample_df_dt = convert_to_datetime(sample_df)

In [ ]:
sample_df_dt.head()

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
0,B-00279,2001-02-19,ΔΕΗ ΑΝΑΝΕΩΣΙΜΕΣ ΑΕ,ΑΔ-00488,2003-03-04,2028-03-04,ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8.0,ΓΕΩΘΕΡΜΙΑ
1,Γ-00798,2004-11-18,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23,2021-03-23,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,45.0,ΑΙΟΛΙΚΑ
2,Γ-01329,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΑΛΙΟΠΥΡΓΟΣ ΑΕ,ΑΔ-00981,2006-10-02,2021-10-02,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ & ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.8,ΑΙΟΛΙΚΑ
3,Γ-01089,2005-07-22,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16,2021-10-16,ΑΤΤΙΚΗΣ,ΝΗΣΩΝ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24.0,ΑΙΟΛΙΚΑ
4,Γ-01328,2006-01-24,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΜΟΝΟΠΡΟΣΩΠΗ ΑΕ,ΑΔ-01257,2009-06-16,2034-06-16,ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.8,ΑΙΟΛΙΚΑ


### Automate the whole thing for various files